In [ ]:
pip install pyspark

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 310.8/310.8 MB 4.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pyspark: filename=pyspark-3.4.0-py2.py3-none-any.whl size=311317145 sha256=47968b4dc81180640d65085ce105e7e4217d2cbe8af0233b75c7e897c02ec35d
  Stored in directory: /root/.cache/pip/wheels/7b/1b/4b/3363a1d04368e7ff0d408e57ff57966fcdf00583774e761327
Successfully built pyspark


In [7]:
pip install -e git+https://github.com/jaybaird/python-bloomfilter.git#egg=pybloom

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Obtaining pybloom from git+https://github.com/jaybaird/python-bloomfilter.git#egg=pybloom
  Cloning https://github.com/jaybaird/python-bloomfilter.git to ./src/pybloom
  Running command git clone --filter=blob:none --quiet https://github.com/jaybaird/python-bloomfilter.git /content/src/pybloom
  Resolved https://github.com/jaybaird/python-bloomfilter.git to commit 2bbe01ad49965bf759e31781e6820408068862ac
  Preparing metadata (setup.py) ... done
  Running setup.py develop for pybloom


In [ ]:
pip install warcio

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.2/40.2 kB 4.4 MB/s eta 0:00:00


In [ ]:
pip install boto3

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 135.6/135.6 kB 8.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.6/79.6 kB 9.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.7/10.7 MB 98.5 MB/s eta 0:00:00


In [1]:
import os
from pyspark import sql as pysql
from pyspark import SparkContext, SparkConf, SQLContext, streaming
from pyspark.sql import functions as psqlf
import pyspark
from pyspark.sql import SparkSession

import pandas as pd
import pybloom
from warcio.archiveiterator import ArchiveIterator
import gzip
import boto3

In [2]:
spark = SparkSession.builder \
    .appName("myApp") \
    .config("spark.driver.memory", "64g") \
    .config("spark.executor.memory", "64g") \
    .getOrCreate()
spark

In [3]:
headers = ['WARC-Type',
    'WARC-Target-URI',
    'WARC-Date',
    'WARC-Record-ID',
    'WARC-Refers-To',
    'WARC-Block-Digest',
    'WARC-Identified-Content-Language',
    'Content-Type',
    'Content-Length'
    ]

In [4]:
ACCESS_KEY=""
SECRET_KEY=""

In [5]:
s3 = boto3.client('s3', aws_access_key_id=ACCESS_KEY, aws_secret_access_key=SECRET_KEY)
obj = s3.get_object(Bucket= 'apbd5254', Key= 'keys_novdec_22.txt')
file = obj['Body']
file = file.read()
file = file.decode("utf-8")
file = file.splitlines()

In [6]:
records = []
for i in range(20):
  obj = s3.get_object(Bucket= 'commoncrawl', Key= file[i])
  with gzip.open(obj['Body'], 'r') as stream:
      for record in ArchiveIterator(stream):
          _record = dict(record.rec_headers.headers)
          _record['Content'] = record.raw_stream.read().decode('utf-8')
          records.append(_record)
  records_df = pd.DataFrame(records)

#drop na on uri columns
records_df = records_df.dropna(axis=0, subset=['WARC-Target-URI'])
records_df['WARC-Identified-Content-Language'] = records_df['WARC-Identified-Content-Language'].fillna('not-available')

spark_df = spark.createDataFrame(records_df)
import urllib
import traceback 

def get_tld(uri):
    try:
        uri_parsed = urllib.parse.urlparse(uri)
        hostname = uri_parsed.hostname
        tld = hostname.split('.')[-1]
        
    except Exception as e:
        print(e)
        print(traceback.format_exc())
        return '.'
    return tld
spark_uris = spark_df.select('WARC-Target-URI')
spark_tlds = spark_uris.rdd.map(lambda a : get_tld(a['WARC-Target-URI']))
spark_tld_counts = spark_tlds.countByValue().items()
spark_tld_counts = sorted(spark_tld_counts, key=(lambda a: a[1]), reverse=True)
s3.put_object(
Body=str(spark_tld_counts), 
Bucket='apbd5254', 
Key='TopLevelDomain/'+'1' +'.txt')

{'ResponseMetadata': {'RequestId': 'A57KF4FGGKAWJ7AY',
  'HostId': 'sZszmqHH78PcBwVr8q1tTCUEE56+u6aC6O8WqeYDUOaWtGdu8i5axR+Xnk2mWgRF04LbOjhYj9UPvkEC6nT1ow==',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'x-amz-id-2': 'sZszmqHH78PcBwVr8q1tTCUEE56+u6aC6O8WqeYDUOaWtGdu8i5axR+Xnk2mWgRF04LbOjhYj9UPvkEC6nT1ow==',
   'x-amz-request-id': 'A57KF4FGGKAWJ7AY',
   'date': 'Tue, 02 May 2023 22:22:25 GMT',
   'x-amz-server-side-encryption': 'AES256',
   'etag': '"fc5f820b3d4f6bab8cb1a5e1471c5874"',
   'server': 'AmazonS3',
   'content-length': '0'},
  'RetryAttempts': 0},
 'ETag': '"fc5f820b3d4f6bab8cb1a5e1471c5874"',
 'ServerSideEncryption': 'AES256'}

In [6]:
records = []
for i in range(20):
  obj = s3.get_object(Bucket= 'commoncrawl', Key= file[i+20])
  with gzip.open(obj['Body'], 'r') as stream:
      for record in ArchiveIterator(stream):
          _record = dict(record.rec_headers.headers)
          _record['Content'] = record.raw_stream.read().decode('utf-8')
          records.append(_record)
  records_df = pd.DataFrame(records)

#drop na on uri columns
records_df = records_df.dropna(axis=0, subset=['WARC-Target-URI'])
records_df['WARC-Identified-Content-Language'] = records_df['WARC-Identified-Content-Language'].fillna('not-available')

spark_df = spark.createDataFrame(records_df)
import urllib
import traceback 

def get_tld(uri):
    try:
        uri_parsed = urllib.parse.urlparse(uri)
        hostname = uri_parsed.hostname
        tld = hostname.split('.')[-1]
        
    except Exception as e:
        print(e)
        print(traceback.format_exc())
        return '.'
    return tld
spark_uris = spark_df.select('WARC-Target-URI')
spark_tlds = spark_uris.rdd.map(lambda a : get_tld(a['WARC-Target-URI']))
spark_tld_counts = spark_tlds.countByValue().items()
spark_tld_counts = sorted(spark_tld_counts, key=(lambda a: a[1]), reverse=True)
s3.put_object(
Body=str(spark_tld_counts), 
Bucket='apbd5254', 
Key='TopLevelDomain/'+'2' +'.txt')

{'ResponseMetadata': {'RequestId': 'N5H3VP2PXNJGH519',
  'HostId': 'wk97nrs0fMcFLUsCz3AQz2l8gCCu1ErdjTZK05OH/Ct6kzDZUffkyiD4C9MYqWpKQ0B384W7MAo=',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'x-amz-id-2': 'wk97nrs0fMcFLUsCz3AQz2l8gCCu1ErdjTZK05OH/Ct6kzDZUffkyiD4C9MYqWpKQ0B384W7MAo=',
   'x-amz-request-id': 'N5H3VP2PXNJGH519',
   'date': 'Tue, 02 May 2023 22:28:09 GMT',
   'x-amz-server-side-encryption': 'AES256',
   'etag': '"4dc663330d68d1ccde956392c0ddd78b"',
   'server': 'AmazonS3',
   'content-length': '0'},
  'RetryAttempts': 0},
 'ETag': '"4dc663330d68d1ccde956392c0ddd78b"',
 'ServerSideEncryption': 'AES256'}

In [6]:
records = []
for i in range(20):
  obj = s3.get_object(Bucket= 'commoncrawl', Key= file[i+40])
  with gzip.open(obj['Body'], 'r') as stream:
      for record in ArchiveIterator(stream):
          _record = dict(record.rec_headers.headers)
          _record['Content'] = record.raw_stream.read().decode('utf-8')
          records.append(_record)
  records_df = pd.DataFrame(records)

#drop na on uri columns
records_df = records_df.dropna(axis=0, subset=['WARC-Target-URI'])
records_df['WARC-Identified-Content-Language'] = records_df['WARC-Identified-Content-Language'].fillna('not-available')

spark_df = spark.createDataFrame(records_df)
import urllib
import traceback 

def get_tld(uri):
    try:
        uri_parsed = urllib.parse.urlparse(uri)
        hostname = uri_parsed.hostname
        tld = hostname.split('.')[-1]
        
    except Exception as e:
        print(e)
        print(traceback.format_exc())
        return '.'
    return tld
spark_uris = spark_df.select('WARC-Target-URI')
spark_tlds = spark_uris.rdd.map(lambda a : get_tld(a['WARC-Target-URI']))
spark_tld_counts = spark_tlds.countByValue().items()
spark_tld_counts = sorted(spark_tld_counts, key=(lambda a: a[1]), reverse=True)
s3.put_object(
Body=str(spark_tld_counts), 
Bucket='apbd5254', 
Key='TopLevelDomain/'+'3' +'.txt')

{'ResponseMetadata': {'RequestId': 'Y7P5S3AQWSHKC3EH',
  'HostId': 'NMr7wWTcDDTW2CkzkDw74VsrVwOW0sx2vlU6DjAVMMSDtBdtgs/qfyI9kOr9cPO0t8Qvk23izFg=',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'x-amz-id-2': 'NMr7wWTcDDTW2CkzkDw74VsrVwOW0sx2vlU6DjAVMMSDtBdtgs/qfyI9kOr9cPO0t8Qvk23izFg=',
   'x-amz-request-id': 'Y7P5S3AQWSHKC3EH',
   'date': 'Tue, 02 May 2023 22:33:37 GMT',
   'x-amz-server-side-encryption': 'AES256',
   'etag': '"a667d5df5219ae5a7e1f467a9dcd9f87"',
   'server': 'AmazonS3',
   'content-length': '0'},
  'RetryAttempts': 0},
 'ETag': '"a667d5df5219ae5a7e1f467a9dcd9f87"',
 'ServerSideEncryption': 'AES256'}

In [6]:
records = []
for i in range(20):
  obj = s3.get_object(Bucket= 'commoncrawl', Key= file[i+60])
  with gzip.open(obj['Body'], 'r') as stream:
      for record in ArchiveIterator(stream):
          _record = dict(record.rec_headers.headers)
          _record['Content'] = record.raw_stream.read().decode('utf-8')
          records.append(_record)
  records_df = pd.DataFrame(records)

#drop na on uri columns
records_df = records_df.dropna(axis=0, subset=['WARC-Target-URI'])
records_df['WARC-Identified-Content-Language'] = records_df['WARC-Identified-Content-Language'].fillna('not-available')

spark_df = spark.createDataFrame(records_df)
import urllib
import traceback 

def get_tld(uri):
    try:
        uri_parsed = urllib.parse.urlparse(uri)
        hostname = uri_parsed.hostname
        tld = hostname.split('.')[-1]
        
    except Exception as e:
        print(e)
        print(traceback.format_exc())
        return '.'
    return tld
spark_uris = spark_df.select('WARC-Target-URI')
spark_tlds = spark_uris.rdd.map(lambda a : get_tld(a['WARC-Target-URI']))
spark_tld_counts = spark_tlds.countByValue().items()
spark_tld_counts = sorted(spark_tld_counts, key=(lambda a: a[1]), reverse=True)
s3.put_object(
Body=str(spark_tld_counts), 
Bucket='apbd5254', 
Key='TopLevelDomain/'+'4' +'.txt')

{'ResponseMetadata': {'RequestId': 'S19QF2DYKHQX7RXX',
  'HostId': 'BCWaWPjxjheZAx309rgGi8gcBiSfUa+UxTPBJaHBD/lgUlVjAkioVcg0fQDbHzRLTY2qOJ9WPX8=',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'x-amz-id-2': 'BCWaWPjxjheZAx309rgGi8gcBiSfUa+UxTPBJaHBD/lgUlVjAkioVcg0fQDbHzRLTY2qOJ9WPX8=',
   'x-amz-request-id': 'S19QF2DYKHQX7RXX',
   'date': 'Tue, 02 May 2023 22:40:04 GMT',
   'x-amz-server-side-encryption': 'AES256',
   'etag': '"a54ad50ed44ee27d496eebde357c728e"',
   'server': 'AmazonS3',
   'content-length': '0'},
  'RetryAttempts': 0},
 'ETag': '"a54ad50ed44ee27d496eebde357c728e"',
 'ServerSideEncryption': 'AES256'}

In [6]:
records = []
for i in range(20):
  obj = s3.get_object(Bucket= 'commoncrawl', Key= file[i+80])
  with gzip.open(obj['Body'], 'r') as stream:
      for record in ArchiveIterator(stream):
          _record = dict(record.rec_headers.headers)
          _record['Content'] = record.raw_stream.read().decode('utf-8')
          records.append(_record)
  records_df = pd.DataFrame(records)

#drop na on uri columns
records_df = records_df.dropna(axis=0, subset=['WARC-Target-URI'])
records_df['WARC-Identified-Content-Language'] = records_df['WARC-Identified-Content-Language'].fillna('not-available')

spark_df = spark.createDataFrame(records_df)
import urllib
import traceback 

def get_tld(uri):
    try:
        uri_parsed = urllib.parse.urlparse(uri)
        hostname = uri_parsed.hostname
        tld = hostname.split('.')[-1]
        
    except Exception as e:
        print(e)
        print(traceback.format_exc())
        return '.'
    return tld
spark_uris = spark_df.select('WARC-Target-URI')
spark_tlds = spark_uris.rdd.map(lambda a : get_tld(a['WARC-Target-URI']))
spark_tld_counts = spark_tlds.countByValue().items()
spark_tld_counts = sorted(spark_tld_counts, key=(lambda a: a[1]), reverse=True)
s3.put_object(
Body=str(spark_tld_counts), 
Bucket='apbd5254', 
Key='TopLevelDomain/'+'5' +'.txt')

{'ResponseMetadata': {'RequestId': 'QXQ8DCED5CCH8B51',
  'HostId': 'zwoToV/PQak13M/RvDEfknzShMSXQLjpvzDtgOyY/hj28ti3WFAplQmOYMBN7353/SIhrvce1Xk=',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'x-amz-id-2': 'zwoToV/PQak13M/RvDEfknzShMSXQLjpvzDtgOyY/hj28ti3WFAplQmOYMBN7353/SIhrvce1Xk=',
   'x-amz-request-id': 'QXQ8DCED5CCH8B51',
   'date': 'Tue, 02 May 2023 22:45:06 GMT',
   'x-amz-server-side-encryption': 'AES256',
   'etag': '"8e2cd215ffd3c52475dc0b5f769027d7"',
   'server': 'AmazonS3',
   'content-length': '0'},
  'RetryAttempts': 0},
 'ETag': '"8e2cd215ffd3c52475dc0b5f769027d7"',
 'ServerSideEncryption': 'AES256'}

In [6]:
records = []
for i in range(20):
  obj = s3.get_object(Bucket= 'commoncrawl', Key= file[i+100])
  with gzip.open(obj['Body'], 'r') as stream:
      for record in ArchiveIterator(stream):
          _record = dict(record.rec_headers.headers)
          _record['Content'] = record.raw_stream.read().decode('utf-8')
          records.append(_record)
  records_df = pd.DataFrame(records)

#drop na on uri columns
records_df = records_df.dropna(axis=0, subset=['WARC-Target-URI'])
records_df['WARC-Identified-Content-Language'] = records_df['WARC-Identified-Content-Language'].fillna('not-available')

spark_df = spark.createDataFrame(records_df)
import urllib
import traceback 

def get_tld(uri):
    try:
        uri_parsed = urllib.parse.urlparse(uri)
        hostname = uri_parsed.hostname
        tld = hostname.split('.')[-1]
        
    except Exception as e:
        print(e)
        print(traceback.format_exc())
        return '.'
    return tld
spark_uris = spark_df.select('WARC-Target-URI')
spark_tlds = spark_uris.rdd.map(lambda a : get_tld(a['WARC-Target-URI']))
spark_tld_counts = spark_tlds.countByValue().items()
spark_tld_counts = sorted(spark_tld_counts, key=(lambda a: a[1]), reverse=True)
s3.put_object(
Body=str(spark_tld_counts), 
Bucket='apbd5254', 
Key='TopLevelDomain/'+'6' +'.txt')

{'ResponseMetadata': {'RequestId': '5DXJAC1MG6A099BC',
  'HostId': 'ZSVwLNNDO0CK0abpF2G65QSBxspyzObwd7/oPU9eHaCWX670wBcxp1i2uQ25MgtQREKcQqIMook=',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'x-amz-id-2': 'ZSVwLNNDO0CK0abpF2G65QSBxspyzObwd7/oPU9eHaCWX670wBcxp1i2uQ25MgtQREKcQqIMook=',
   'x-amz-request-id': '5DXJAC1MG6A099BC',
   'date': 'Tue, 02 May 2023 22:50:05 GMT',
   'x-amz-server-side-encryption': 'AES256',
   'etag': '"5d2891ce47d628ec3661bc30aa973ea8"',
   'server': 'AmazonS3',
   'content-length': '0'},
  'RetryAttempts': 0},
 'ETag': '"5d2891ce47d628ec3661bc30aa973ea8"',
 'ServerSideEncryption': 'AES256'}

In [6]:
records = []
for i in range(20):
  obj = s3.get_object(Bucket= 'commoncrawl', Key= file[i+120])
  with gzip.open(obj['Body'], 'r') as stream:
      for record in ArchiveIterator(stream):
          _record = dict(record.rec_headers.headers)
          _record['Content'] = record.raw_stream.read().decode('utf-8')
          records.append(_record)
  records_df = pd.DataFrame(records)

#drop na on uri columns
records_df = records_df.dropna(axis=0, subset=['WARC-Target-URI'])
records_df['WARC-Identified-Content-Language'] = records_df['WARC-Identified-Content-Language'].fillna('not-available')

spark_df = spark.createDataFrame(records_df)
import urllib
import traceback 

def get_tld(uri):
    try:
        uri_parsed = urllib.parse.urlparse(uri)
        hostname = uri_parsed.hostname
        tld = hostname.split('.')[-1]
        
    except Exception as e:
        print(e)
        print(traceback.format_exc())
        return '.'
    return tld
spark_uris = spark_df.select('WARC-Target-URI')
spark_tlds = spark_uris.rdd.map(lambda a : get_tld(a['WARC-Target-URI']))
spark_tld_counts = spark_tlds.countByValue().items()
spark_tld_counts = sorted(spark_tld_counts, key=(lambda a: a[1]), reverse=True)
s3.put_object(
Body=str(spark_tld_counts), 
Bucket='apbd5254', 
Key='TopLevelDomain/'+'7' +'.txt')

{'ResponseMetadata': {'RequestId': 'R458HCY3Q6B3E7HW',
  'HostId': 'vqOtagHxupIkq+OQbwbNZ0OZlLcul58u4W2e8urJckaIrBmrS5QalAkNctFyVr+ZDawJoWbp1Vk=',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'x-amz-id-2': 'vqOtagHxupIkq+OQbwbNZ0OZlLcul58u4W2e8urJckaIrBmrS5QalAkNctFyVr+ZDawJoWbp1Vk=',
   'x-amz-request-id': 'R458HCY3Q6B3E7HW',
   'date': 'Tue, 02 May 2023 22:56:24 GMT',
   'x-amz-server-side-encryption': 'AES256',
   'etag': '"795a102f7229138c804d205ab9fdc489"',
   'server': 'AmazonS3',
   'content-length': '0'},
  'RetryAttempts': 0},
 'ETag': '"795a102f7229138c804d205ab9fdc489"',
 'ServerSideEncryption': 'AES256'}

In [6]:
records = []
for i in range(20):
  obj = s3.get_object(Bucket= 'commoncrawl', Key= file[i+140])
  with gzip.open(obj['Body'], 'r') as stream:
      for record in ArchiveIterator(stream):
          _record = dict(record.rec_headers.headers)
          _record['Content'] = record.raw_stream.read().decode('utf-8')
          records.append(_record)
  records_df = pd.DataFrame(records)

#drop na on uri columns
records_df = records_df.dropna(axis=0, subset=['WARC-Target-URI'])
records_df['WARC-Identified-Content-Language'] = records_df['WARC-Identified-Content-Language'].fillna('not-available')

spark_df = spark.createDataFrame(records_df)
import urllib
import traceback 

def get_tld(uri):
    try:
        uri_parsed = urllib.parse.urlparse(uri)
        hostname = uri_parsed.hostname
        tld = hostname.split('.')[-1]
        
    except Exception as e:
        print(e)
        print(traceback.format_exc())
        return '.'
    return tld
spark_uris = spark_df.select('WARC-Target-URI')
spark_tlds = spark_uris.rdd.map(lambda a : get_tld(a['WARC-Target-URI']))
spark_tld_counts = spark_tlds.countByValue().items()
spark_tld_counts = sorted(spark_tld_counts, key=(lambda a: a[1]), reverse=True)
s3.put_object(
Body=str(spark_tld_counts), 
Bucket='apbd5254', 
Key='TopLevelDomain/'+'8' +'.txt')

{'ResponseMetadata': {'RequestId': 'WEEYBKN1Y6F7RVSM',
  'HostId': '5o1cmYhS2qbhq3tksREwn6mzGx2LDd7pkzteSC2euLmntd/SRAsbCY00YRjT2mBOVyZVZMbMMjk=',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'x-amz-id-2': '5o1cmYhS2qbhq3tksREwn6mzGx2LDd7pkzteSC2euLmntd/SRAsbCY00YRjT2mBOVyZVZMbMMjk=',
   'x-amz-request-id': 'WEEYBKN1Y6F7RVSM',
   'date': 'Tue, 02 May 2023 23:01:20 GMT',
   'x-amz-server-side-encryption': 'AES256',
   'etag': '"7a8cec81729a46e2c234f67f2ac4daec"',
   'server': 'AmazonS3',
   'content-length': '0'},
  'RetryAttempts': 0},
 'ETag': '"7a8cec81729a46e2c234f67f2ac4daec"',
 'ServerSideEncryption': 'AES256'}

In [6]:
records = []
for i in range(20):
  obj = s3.get_object(Bucket= 'commoncrawl', Key= file[i+160])
  with gzip.open(obj['Body'], 'r') as stream:
      for record in ArchiveIterator(stream):
          _record = dict(record.rec_headers.headers)
          _record['Content'] = record.raw_stream.read().decode('utf-8')
          records.append(_record)
  records_df = pd.DataFrame(records)

#drop na on uri columns
records_df = records_df.dropna(axis=0, subset=['WARC-Target-URI'])
records_df['WARC-Identified-Content-Language'] = records_df['WARC-Identified-Content-Language'].fillna('not-available')

spark_df = spark.createDataFrame(records_df)
import urllib
import traceback 

def get_tld(uri):
    try:
        uri_parsed = urllib.parse.urlparse(uri)
        hostname = uri_parsed.hostname
        tld = hostname.split('.')[-1]
        
    except Exception as e:
        print(e)
        print(traceback.format_exc())
        return '.'
    return tld
spark_uris = spark_df.select('WARC-Target-URI')
spark_tlds = spark_uris.rdd.map(lambda a : get_tld(a['WARC-Target-URI']))
spark_tld_counts = spark_tlds.countByValue().items()
spark_tld_counts = sorted(spark_tld_counts, key=(lambda a: a[1]), reverse=True)
s3.put_object(
Body=str(spark_tld_counts), 
Bucket='apbd5254', 
Key='TopLevelDomain/'+'9' +'.txt')

{'ResponseMetadata': {'RequestId': '14NVN5PMCW1B7XEJ',
  'HostId': 'LxoHLRc9WcXzqFBKsU/hbvuRWxF0c63TLcsq3nT7yrcxE0kJixv4WzgMWG1wV930ByqmX8/Ckto=',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'x-amz-id-2': 'LxoHLRc9WcXzqFBKsU/hbvuRWxF0c63TLcsq3nT7yrcxE0kJixv4WzgMWG1wV930ByqmX8/Ckto=',
   'x-amz-request-id': '14NVN5PMCW1B7XEJ',
   'date': 'Tue, 02 May 2023 23:06:16 GMT',
   'x-amz-server-side-encryption': 'AES256',
   'etag': '"23d32661fb546fb8696275e4b5142ded"',
   'server': 'AmazonS3',
   'content-length': '0'},
  'RetryAttempts': 0},
 'ETag': '"23d32661fb546fb8696275e4b5142ded"',
 'ServerSideEncryption': 'AES256'}

In [6]:
records = []
for i in range(20):
  obj = s3.get_object(Bucket= 'commoncrawl', Key= file[i+180])
  with gzip.open(obj['Body'], 'r') as stream:
      for record in ArchiveIterator(stream):
          _record = dict(record.rec_headers.headers)
          _record['Content'] = record.raw_stream.read().decode('utf-8')
          records.append(_record)
  records_df = pd.DataFrame(records)

#drop na on uri columns
records_df = records_df.dropna(axis=0, subset=['WARC-Target-URI'])
records_df['WARC-Identified-Content-Language'] = records_df['WARC-Identified-Content-Language'].fillna('not-available')

spark_df = spark.createDataFrame(records_df)
import urllib
import traceback 

def get_tld(uri):
    try:
        uri_parsed = urllib.parse.urlparse(uri)
        hostname = uri_parsed.hostname
        tld = hostname.split('.')[-1]
        
    except Exception as e:
        print(e)
        print(traceback.format_exc())
        return '.'
    return tld
spark_uris = spark_df.select('WARC-Target-URI')
spark_tlds = spark_uris.rdd.map(lambda a : get_tld(a['WARC-Target-URI']))
spark_tld_counts = spark_tlds.countByValue().items()
spark_tld_counts = sorted(spark_tld_counts, key=(lambda a: a[1]), reverse=True)
s3.put_object(
Body=str(spark_tld_counts), 
Bucket='apbd5254', 
Key='TopLevelDomain/'+'10' +'.txt')

{'ResponseMetadata': {'RequestId': 'DEBGRVQA66XAW9S9',
  'HostId': 'sqRWQTbDA797HV4qjPneNM58Abafzi8IhrtXRYKs0b8YJxOjyaG4R3ODlhkQQ/jlZrQDbFPnVHlW+IB6OGTpcstefGib8m+T/dYk9MAX9G4=',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'x-amz-id-2': 'sqRWQTbDA797HV4qjPneNM58Abafzi8IhrtXRYKs0b8YJxOjyaG4R3ODlhkQQ/jlZrQDbFPnVHlW+IB6OGTpcstefGib8m+T/dYk9MAX9G4=',
   'x-amz-request-id': 'DEBGRVQA66XAW9S9',
   'date': 'Tue, 02 May 2023 23:11:14 GMT',
   'x-amz-server-side-encryption': 'AES256',
   'etag': '"34f6069aca62a5f9b31919f3a7da3cb2"',
   'server': 'AmazonS3',
   'content-length': '0'},
  'RetryAttempts': 0},
 'ETag': '"34f6069aca62a5f9b31919f3a7da3cb2"',
 'ServerSideEncryption': 'AES256'}